In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
np.random.seed=1994

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
pd.set_option('display.max_rows', 500)

# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

In [0]:

train=pd.read_csv('/content/gdrive/My Drive/Competetion_ML/Zindi/Train.csv')
test=pd.read_csv('/content/gdrive/My Drive/Competetion_ML/Zindi/Test.csv')
s=pd.read_csv('/content/gdrive/My Drive/Competetion_ML/Zindi/SampleSubmission (1).csv')

In [0]:
# df=train.append(test,ignore_index=True)
# df.head()

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,...,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92.0,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,38.622451,-61.789016,...,0.000000,76.536426,38.593017,-61.752587,22.363665,0.000064,0.566828,NaN,NaN,0.000000,NaN,NaN,NaN,76.536426,38.593017,-61.752587,22.363665,NaN,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.000040,-1.861476,0.000000,76.536426,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91.0,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.022017,1148.985447,61.216687,841116.763051,-57.015200,61.402626,-74.457583,...,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.000171,0.858446,175.019862,99354.162958,0.059358,5.958538,175.072145,99353.672374,-14.708036,59.624912,-67.693509,28.614804,0.315403,-1.082553,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000150,0.433957,0.000050,-1.452612,0.059433,-14.708036,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96.0,16.400000,33.400002,0.00750,27.031030,5.065727,3.500559,0.000067,0.000170,-1.001242,0.015904,841410.713456,-

In [0]:
df=train.append(test,ignore_index=True)
df.head()

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,...,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92.0,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,38.622451,-61.789016,...,0.000000,76.536426,38.593017,-61.752587,22.363665,0.000064,0.566828,NaN,NaN,0.000000,NaN,NaN,NaN,76.536426,38.593017,-61.752587,22.363665,NaN,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.000040,-1.861476,0.000000,76.536426,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91.0,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.022017,1148.985447,61.216687,841116.763051,-57.015200,61.402626,-74.457583,...,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.000171,0.858446,175.019862,99354.162958,0.059358,5.958538,175.072145,99353.672374,-14.708036,59.624912,-67.693509,28.614804,0.315403,-1.082553,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000150,0.433957,0.000050,-1.452612,0.059433,-14.708036,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96.0,16.400000,33.400002,0.00750,27.031030,5.065727,3.500559,0.000067,0.000170,-1.001242,0.015904,841410.713456,-

In [0]:

def create_features(df, label=None,seg=None):
    """
    Creates time series features from datetime index.
    """
    df = df.copy()

    df['date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['week'] = df['date'].dt.week
    df['is_month_start']=df['date'].dt.is_month_start
    df['is_month_end']=df['date'].dt.is_month_end
    df['is_quarter_start']=df['date'].dt.is_quarter_start
    df['is_quarter_end']=df['date'].dt.is_quarter_end
    df['is_year_start']=df['date'].dt.is_year_start
    df['is_year_end']=df['date'].dt.is_year_end
    df['Semester'] = np.where(df['quarter'].isin([1,2]),1,2)
    df['Date']=pd.factorize(df['Date'])[0]


    df['Place_ID']=df['Place_ID'].astype('category')
    l=['precipitable_water_entire_atmosphere','relative_humidity_2m_above_ground',
                                                        'specific_humidity_2m_above_ground','temperature_2m_above_ground','u_component_of_wind_10m_above_ground',
                                                        'v_component_of_wind_10m_above_ground']

    df[[x+'cumsum' for x in l]]=df.sort_values('Date').groupby('Place_ID').cumsum()[l].sort_index()
    df[[x+'diff' for x in l]]=df.sort_values('Date').groupby('Place_ID')[l].diff().sort_index()
    return df.drop(['date'],axis=1)

In [0]:

d={}
for k in [x for x in df.columns if x not in ['Place_ID X Date', 'Date', 'Place_ID', 'target', 'target_min',
      'target_max', 'target_variance', 'target_count']]:
  d[k]=['min','max','sum','mean','std']


# t1 = df.groupby('Place_ID').agg(d)
# t1.columns=['LC_' + '_'.join(col).strip() for col in t1.columns.values]
# t1.reset_index(inplace=True)
# df = df.merge(t1,on='Place_ID',how='left')
t1 = df.groupby('Date').agg(d)
t1.columns=['LD_' + '_'.join(col).strip() for col in t1.columns.values]
t1.reset_index(inplace=True)
df = df.merge(t1,on='Date',how='left')

In [0]:
df=create_features(df)
df.shape

(46693, 478)

In [0]:
df_train=df[df['target'].isnull()==False].copy()
df_test=df[df['target'].isnull()==True].copy()

print(df_train.shape,df_test.shape)

(30557, 478) (16136, 478)


In [0]:
from lightgbm import LGBMClassifier,LGBMRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error


In [0]:
features=[x for x in df.columns if x not in ['target', 'target_count', 'target_max', 'target_min','target_variance','Place_ID']]

In [0]:
X,y=df_train[features].drop(['Place_ID X Date'],axis=1),df_train['target']
Xtest=df_test[features].drop(['Place_ID X Date'],axis=1)
y_bin = pd.qcut(y,10,labels=False,duplicates='drop')
print(y_bin)


print(X.shape,Xtest.shape)
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state = 1994,stratify=y_bin)

0        3
1        4
2        2
3        4
4        1
        ..
30552    1
30553    5
30554    7
30555    8
30556    7
Name: target, Length: 30557, dtype: int64
(30557, 471) (16136, 471)


In [0]:
# s.to_csv('colabtry4.csv',index=False) ,reg_alpha=1,reg_lambda=10
# s.target.describe()
err=[]
y_pred_tot=[]
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1994)
i=1
for train_index, test_index in fold.split(X,y_bin):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=LGBMRegressor(n_estimators=10000,random_state=1994,learning_rate=0.03,num_leaves=50,boosting_type='gbdt',feature_fraction=0.6,
                objective='regression',min_child_samples=12)
    m.fit(X_train,y_train,eval_set=[(X_test, y_test)],eval_metric='rmse', early_stopping_rounds=200,verbose=400)
    preds=m.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    err.append(np.sqrt(mean_squared_error(y_test,preds)))
    p = m.predict(Xtest)
    i=i+1
    y_pred_tot.append(p)

Training until validation scores don't improve for 200 rounds.
[400]	valid_0's rmse: 27.6809	valid_0's l2: 766.234
[800]	valid_0's rmse: 26.6669	valid_0's l2: 711.122
[1200]	valid_0's rmse: 26.1398	valid_0's l2: 683.291
[1600]	valid_0's rmse: 25.8237	valid_0's l2: 666.864
[2000]	valid_0's rmse: 25.6315	valid_0's l2: 656.972
[2400]	valid_0's rmse: 25.5072	valid_0's l2: 650.619
[2800]	valid_0's rmse: 25.4322	valid_0's l2: 646.795
[3200]	valid_0's rmse: 25.3661	valid_0's l2: 643.437
[3600]	valid_0's rmse: 25.3048	valid_0's l2: 640.331
[4000]	valid_0's rmse: 25.2527	valid_0's l2: 637.699
[4400]	valid_0's rmse: 25.2198	valid_0's l2: 636.038
[4800]	valid_0's rmse: 25.1916	valid_0's l2: 634.617
[5200]	valid_0's rmse: 25.1672	valid_0's l2: 633.386
[5600]	valid_0's rmse: 25.1411	valid_0's l2: 632.077
[6000]	valid_0's rmse: 25.1218	valid_0's l2: 631.105
[6400]	valid_0's rmse: 25.1165	valid_0's l2: 630.837
[6800]	valid_0's rmse: 25.1088	valid_0's l2: 630.452
[7200]	valid_0's rmse: 25.1014	valid_0

In [0]:
# # s.to_csv('colabtry4.csv',index=False) ,reg_alpha=1,reg_lambda=10
# # s.target.describe()
# err=[]
# y_pred_tot=[]
# from sklearn.model_selection import KFold,StratifiedKFold
# fold=StratifiedKFold(n_splits=5,shuffle=True,random_state=1994)
# i=1
# for train_index, test_index in fold.split(X,y_bin):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     m=LGBMRegressor(n_estimators=10000,random_state=1994,learning_rate=0.03,num_leaves=50,boosting_type='gbdt',feature_fraction=0.6,
#                 objective='regression',min_child_samples=12)
#     m.fit(X_train,y_train,eval_set=[(X_test, y_test)],eval_metric='rmse', early_stopping_rounds=200,verbose=400)
#     preds=m.predict(X_test)
#     print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
#     err.append(np.sqrt(mean_squared_error(y_test,preds)))
#     p = m.predict(Xtest)
#     i=i+1
#     y_pred_tot.append(p)

In [0]:
np.mean(err)

24.22103461016549

In [0]:
s['target']=np.mean(y_pred_tot,0)
s

,Place_ID X Date,target
0,0OS9LVX X 2020-01-02,46.074689
1,0OS9LVX X 2020-01-03,35.531098
2,0OS9LVX X 2020-01-04,32.588001
3,0OS9LVX X 2020-01-05,36.428560
4,0OS9LVX X 2020-01-06,33.887611
...,...,...
16131,ZZDJZMS X 2020-03-31,35.300739
16132,ZZDJZMS X 2020-04-01,47.987413
16133,ZZDJZMS X 2020-04-02,50.178225
16134,ZZDJZMS X 2020-04-03,36.965615


In [0]:
s.to_csv('zindi_colab_try2_lgbm2.csv',index=False)

In [0]:
from google.colab import files
files.download('zindi_colab_try2_lgbm2.csv') 